# 2. Leaky Aquifer Test - Hardinxveld 

### Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import timflow.transient as tft

plt.rcParams["figure.figsize"] = [5, 3]

### Introduction and Conceptual Model

Consider a pumping test conducted near the town of Hardinxveld-Giessendam in the Netherlands in 1981. The objective was to quantify the aquifer parameters and the head-loss at the pumping well caused by clogging.
The subsurface consists of a 10 meter thick aquitard, followed by a 27 m aquifer. Below the aquifer is another aquitard of 31 m thickness and a 20 m thick second aquifer. 
Five pumping wells are screened in the first aquifer. Drawdown measurements at of one of wells is obtained from the MLU documentation (Carlson & Randall, 2012). The pumping well was operated for 20 minutes at 1848 m$^3$/d. Drawdown was measured inside the pumping well during pumping and during 30 minutes of recovery. The radius of the pumped well is 0.155 m. Two `timflow` models are calibrated. The first one has an impermeable top and bottom. For the second model, the effect of the semi-confining top and the aquitard and aquifer below the pumped aquifer are combined into one semi-confining layer. By trial and error, the resistance of this semi-confining layer was set to $c=2000$ d. For both models, three parameters are calibrated: the hydraulic conductivity and specific storage of the aquifer and the skin resistance of the well. 

<img src="./figs/Hardinxveld.png" style="width:400pt">

### Load data

In [ ]:
data = np.loadtxt("data/recovery.txt", skiprows=1)
to = data[:, 0]
ho = data[:, 1]

### Parameters and model

In [ ]:
# known parameters
H = 27  # aquifer thickness, m
zt = -10  # upper boundary of aquifer, m
zb = zt - H  # lower boundary of the aquifer, m
rw = 0.155  # well screen radius, m
Q = 1848  # constant discharge rate, m^3/d
tstop = 0.013889  # time at which pumping stops, d

In [ ]:
# model with impermeable top and bottom
ml = tft.ModelMaq(
    kaq=[50],
    z=[zt, zb],
    Saq=[1e-4],
    topboundary="conf",
    tmin=1e-4,
    tmax=1,
)
w = tft.Well(ml, xw=0, yw=0, rw=rw, res=1, tsandQ=[(0, Q), (tstop, 0)], layers=0)
ml.solve()

In [ ]:
cal = tft.Calibrate(ml)
cal.set_parameter(name="kaq", initial=50, pmin=0, layers=0)
cal.set_parameter(name="Saq", initial=1e-4, pmin=0, pmax=1e-3, layers=0)

cal.set_parameter_by_reference(name="res", parameter=w.res[:], initial=1, pmin=0)
cal.seriesinwell(name="obs", element=w, t=to, h=ho)
cal.fit(report=False)

In [ ]:
display(cal.parameters["optimal"])
print(f"RMSE: {cal.rmse():.5f} m")

In [ ]:
# model with semi-confining top
ml1 = tft.ModelMaq(
    kaq=[50],
    z=[0, zt, zb],
    Saq=[1e-4],
    c=2000,
    topboundary="semi",
    tmin=1e-4,
    tmax=1,
)
w1 = tft.Well(ml1, xw=0, yw=0, rw=rw, res=1, tsandQ=[(0, Q), (tstop, 0)], layers=0)
ml1.solve()

In [ ]:
cal1 = tft.Calibrate(ml1)
cal1.set_parameter(name="kaq", initial=50, pmin=0, layers=0)
cal1.set_parameter(name="Saq", initial=1e-4, pmin=0, pmax=1e-3, layers=0)

cal1.set_parameter_by_reference(name="res", parameter=w1.res[:], initial=1, pmin=0)
cal1.seriesinwell(name="obs", element=w1, t=to, h=ho)
cal1.fit(report=False)

In [ ]:
display(cal1.parameters["optimal"])
print(f"RMSE: {cal1.rmse():.5f} m")

In [ ]:
tm = np.logspace(np.log10(to[0]), np.log10(to[-1]), 100)
h = w.headinside(tm)
h1 = w1.headinside(tm)
plt.loglog(to, -ho, ".", label="observed")
plt.loglog(tm, -h[0], label="timflow confined")
plt.loglog(tm, -h1[0], label="timflow semi-confined")
plt.xlabel("time [d]")
plt.ylabel("drawdown [m]")
plt.legend()
plt.title("Model Results")
plt.grid()

### Comparison of results

The performance of `timflow` is compared to MLU (Carlson and Randall, 2012). The MLU parameters are similar to the `timflow` parameters for the confined case. The semi-confined case gives somewhat lower RMSE and a slightly better fit. It is noted that this model is exceedingly sensitive to the specified moment that the well is turned off. Even a change of 1 second will produce significantly different results. 

In [ ]:
t = pd.DataFrame(
    columns=["k [m/d]", "Ss [1/m]", "res", "RMSE [m]"],
    index=["timflow confined", "timflow semi-confined", "MLU"],
)

t.loc["timflow confined"] = np.append(cal.parameters["optimal"].values, cal.rmse())
t.loc["timflow semi-confined"] = np.append(cal1.parameters["optimal"].values, cal1.rmse())
t.loc["MLU"] = [51.530, 8.16e-04, 0.022, 0.00756]

t_formatted = t.style.format(
    {"k [m/d]": "{:.2f}", "Ss [1/m]": "{:.2e}", "res": "{:.3f}", "RMSE [m]": "{:.4f}"}
)
t_formatted

## References

* Carlson F. and Randall J. (2012), MLU: a Windows application for the analysis of aquifer tests and the design of well fields in layered systems, Ground Water 50(4):504–510
* Duffield, G.M. (2007), AQTESOLV for Windows Version 4.5 User's Guide, HydroSOLVE, Inc., Reston, VA.
* Newville, M., Stensitzki, T., Allen, D.B. and Ingargiola, A. (2014), LMFIT: Non Linear Least-Squares Minimization and Curve Fitting for Python, https://dx.doi.org/10.5281/zenodo.11813, https://lmfit.github.io/lmfit-py/intro.html (last access: August,2021).